In [ ]:
from retinaface import RetinaFace
from feat.detector import Detector
import pandas as pd
import matplotlib.pyplot as plt

import sys
import import_ipynb
from Image_Utilities import readImage, writeImage, deleteFile, showImage, getImageCrop

In [ ]:
class FD:
    
    """
    This class implements the facial detection and emotions detection algorithm
    """
    
    def __init__(self, args):
        """
        FD class constructor
        """
        
        self.args            = args
        self.detector        = None
        
        if "image_path" in args:
            self.image           = readImage(args["image_path"])        
        
        if "coords" in args:
            if args["coords"] != None:
                self.coords          = args["coords"]
                print(self.coords)
    
    def initDetector(self):
        """
        Initialize PyFeat detector
        """
        self.detector = Detector(
            face_model       = self.args["face_model"],
            landmark_model   = self.args["landmark_model"],
            au_model         = self.args["au_model"],
            emotion_model    = self.args["emotion_model"],
            facepose_model   = self.args["facepose_model"],
            pose_model       = self.args["pose_model"]
        )
        print("detector initialized")
    
    def facialDetection(self):
        """
        Detect faces of the current image
        """
        self.faceCrops = []
        for c in self.coords:
            print("COORDS: ",c)
            personCrop = getImageCrop(self.image, c)
            obj = RetinaFace.detect_faces(personCrop)
            if "face_1" in obj:
                faceCrop = getImageCrop(personCrop, obj['face_1']['facial_area'])
                self.faceCrops.append(faceCrop)
                if self.args["debug"]:
                    showImage(personCrop)
                    showImage(faceCrop)

        
    def checkImages(self):
        """
        Save faces as images
        """
        i = 0
        for f in self.faceCrops:
            writeImage(self.args["face_folder"]+str(i)+"FaceOf"+self.args["image_name"],f,False)
            i = i+1   
            
            
    def perfornmDetection(self):
        """
        Perform emotions detections on the current detected faces
        """
        emotionsTotal = self.args['emotions']
        for e in emotionsTotal:
            emotionsTotal[e] = 0
        returnBool = True
        if self.args["debug"]:
            print("Debug on")
        for i in range(len(self.faceCrops)):
            restart = False
            faceEmotions, restart = self.detection(self.args["face_folder"]+str(i)+"FaceOf"+self.args["image_name"], i)
            if restart:
                self.facialDetection()
                self.checkImages()
                result = self.perfornmDetection() 
                print(result)
                return result
            if faceEmotions is not None:
                returnBool = False
                emotionsTotal = self.addUpEmotions(emotionsTotal, faceEmotions)
        
        if returnBool:
            if self.args["debug"]:
                print("NONE NONE")
            return None, None
        else: 
            if self.args["debug"]:
                print(self.averageEmotions(emotionsTotal), len(self.faceCrops) == 1)
            return self.averageEmotions(emotionsTotal), len(self.faceCrops) == 1
    
    def detection(self, imagePath, i):
        """
        Perform emotion detection on the current face
        """
        if self.detector is None:
            print("ERROR: detector not initialized")
            return 0
        
        try:
            detection =  self.detector.detect_image(imagePath)
            detection
            detection.facebox
            if self.args["debug"]:
                print(detection.aus)
                print(detection.emotions)
                detection.plot_detections(poses=True)
            return detection.emotions, False
        except:
            print("An exception happened when processing the face",i,"of the image "+imagePath)
            print("ERROR WHEN TRYING TO THE DETECT A FACE IN:", self.coords)
            answer = input("Switch coords/next/exit?(enter/n/q)")
            if answer == "q":
                sys.exit()
            elif answer == "n":
                return None, False
            else:
                newCoords = input("Coords coma separated:").split(",")
                self.coords = [[int(i) for i in newCoords]]
                return None, True
                
        return None, False
    
    
    def addUpEmotions(self, emotionsTotal, faceEmotions):
        """
        Add emotions values
        """
        emotionsTotal['count'] = emotionsTotal['count']+1
        emotionsTotal['anger'] = emotionsTotal['anger'] + faceEmotions.at[0, 'anger']
        emotionsTotal['disgust'] = emotionsTotal['disgust'] + faceEmotions.at[0, 'disgust']
        emotionsTotal['fear'] = emotionsTotal['fear'] + faceEmotions.at[0, 'fear']
        emotionsTotal['happiness'] = emotionsTotal['happiness'] + faceEmotions.at[0, 'happiness']
        emotionsTotal['sadness'] = emotionsTotal['sadness'] + faceEmotions.at[0, 'sadness']
        emotionsTotal['surprise'] = emotionsTotal['surprise'] + faceEmotions.at[0, 'surprise']
        emotionsTotal['neutral'] = emotionsTotal['neutral'] + faceEmotions.at[0, 'neutral']
        
        print(emotionsTotal)
        
        return emotionsTotal
    
    def averageEmotions(self, emotionsTotal):
        """
        Calculate the emotions average
        """
        averageEmotions = {}
        averageEmotions['anger'] = emotionsTotal['anger'] / emotionsTotal['count']
        averageEmotions['disgust'] = emotionsTotal['disgust'] / emotionsTotal['count']
        averageEmotions['fear'] = emotionsTotal['fear'] / emotionsTotal['count']
        averageEmotions['happiness'] = emotionsTotal['happiness'] / emotionsTotal['count']
        averageEmotions['sadness'] = emotionsTotal['sadness'] / emotionsTotal['count']
        averageEmotions['surprise'] = emotionsTotal['surprise'] / emotionsTotal['count']
        averageEmotions['neutral'] = emotionsTotal['neutral'] / emotionsTotal['count']
        
        return averageEmotions
    
    def setImage(self, imagePath):
        """
        This method set the current image
        """
        self.image = readImage(imagePath)
        
    def setCoords(self, coords):
        """
        This method set the current coords
        """
        self.coords = coords

In [ ]:
# if __name__ == "__main__":
    
#     from Args import fdArguments
    
#     args = fdArguments()
    
#     fd = FD(args)
#     fd.initDetector()
#     fd.setCoords([[2861, 1896, 411, 732]])
#     fd.setImage("./TransGranCanaria2020/Images/00002transgrancanaria2020-26025.JPG")
#     fd.facialDetection()
#     fd.checkImages()
#     averageEmotions = fd.perfornmDetection()
#     print(averageEmotions)
    